In [2]:
import requests
import urllib
import json
from urllib.parse import urlparse
import pprint

api_key = "Enter your API key here"  #### API-key

file = open('dests.txt','r',encoding ='utf-8')
file = file.readlines() 

dic = {}
Most3FarCities = {}
final_dict = {}
components_tup = tuple()

for line in file:
    dic["origins"] = 'תל אביב'
    dic["destinations"] = line
    dic["key"] = api_key
        
    ##distancematrix api
    URLdistancematrix = 'https://maps.googleapis.com/maps/api/distancematrix/json?'
    url = URLdistancematrix + urllib.parse.urlencode(dic)
    try:
        response_distancematrix = requests.get(url)
        if response_distancematrix.status_code != 200:
            print("HTTP error ", response_distancematrix.status_code)
    except:
        print("Something went wrong with requests.get()")
        break
    try:
        response_distancematrix = response_distancematrix.json()
    except:
        print("Response is not in valid json format")
        break
    response_distancematrix = json.dumps(response_distancematrix)
    response_distancematrix = json.loads(response_distancematrix)
    try:
        distance = response_distancematrix["rows"][0]["elements"][0]["distance"]["text"]
        duration = response_distancematrix["rows"][0]["elements"][0]["duration"]["text"]
        
        #geocoding api
        URLgeocode ="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (line,api_key)
        response_geocode = requests.get(URLgeocode).json()
        response_geocode = json.dumps(response_geocode)
        response_geocode = json.loads(response_geocode)
        longitude = response_geocode["results"][0]["geometry"]["location"]["lng"]
        latitude = response_geocode["results"][0]["geometry"]["location"]["lat"]
        
        Most3FarCities[response_distancematrix["destination_addresses"][0]] = distance
        components_tup = ("distance: "+distance, "duration: "+str(duration), "longitude: "+str(longitude), "latitude: "+str(latitude))
        final_dict[response_distancematrix["destination_addresses"][0]] = (components_tup)
    except:
        print("The destination: ",line.rstrip(),"doesn't exist. Please enter a new one")
        break

listOftuple = []
for key,value in sorted(Most3FarCities.items()):
    new_tup = (value, key)
    listOftuple.append(new_tup)
listOftuple = sorted(listOftuple, reverse=True)

if len(final_dict)==5:
    pprint.pprint(final_dict) ##printing the dictionary with its components
    print("\n")
    print("The three farthest cities from Tel Aviv are:")
    print(listOftuple[:3]) ##printing the farthest cities from Tel Aviv
else:
    print("Your file doesn't have 5 destinations. Please enter a new file with 5 destinations.")

The destination:  איסטנבול doesn't exist. Please enter a new one
Your file doesn't have 5 destinations. Please enter a new file with 5 destinations.
